In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from permchain.langgraph import Actor, DecisionPoint, Graph

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

from langchain_core.agents import AgentFinish
# Define decision-making logic
def should_continue(data):
    # Logic to decide whether to continue in the loop or exit
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"
    
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data
    
    

# Define agents
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define a new graph
workflow = Graph()
decision_point = DecisionPoint("exit?", should_continue)
llm_agent = Actor("agent", agent)
tool_actor = Actor("tools", RunnableLambda(execute_tools))

# Register actors
workflow.register(llm_agent)
workflow.register(tool_actor)
workflow.register(decision_point)

# Define connections with conditional logic
workflow.connect(llm_agent, decision_point)
workflow.branch(decision_point, tool_actor, condition="continue")
workflow.branch(decision_point, None, condition="exit")  # Exit the workflow
workflow.connect(tool_actor, llm_agent)

# Define entry point and execute the graph
workflow.set_entry_point(llm_agent)
chain = workflow.compile()

chain.invoke({"input": "what is the weather in sf", "intermediate_steps": []})